# 보안2 과정 함수화

1. 이미지 매칭
 - 평균 해시 매칭(Average Hash Matching)

2. 특징 디스크립터 검출기와 올바른 매칭점 찾기
 - 디스크립터 검출기: ORB (Oriented and Rotated BRIEF)
 - 매칭점 찾기 평가 지표: knnMatch()


#### 라이브러리

In [2]:

# Data handling
import numpy as np


# OpenCV, OCR
import cv2
import pytesseract
from skimage.transform import resize


# Youtube video
import pafy
import cv2


# Visualizatipn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', family='malgun gothic')
plt.rc('axes', unicode_minus=False)

# time
from time import time

#### 차량 진입 사진 로드

In [3]:
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype) 
        img = cv2.imdecode(n, flags) 
        return img 
    except Exception as e: 
        print(e) 
        return None

In [4]:
greencar01 = imread('./car_image/그린카진입01.jpg')
greencar02 = imread('./car_image/그린카진입02.jpg')
socar01 = imread('./car_image/쏘카진입01.jpg')
socar02 = imread('./car_image/쏘카진입02.jpg')

# 1. 이미지 매칭
 - 평균 해시 매칭(Average Hash Matching)

In [16]:
greencar01_hash = img2hash(greencar01)
greencar02_hash = img2hash(greencar02)
socar01_hash = img2hash(socar01)
socar02_hash = img2hash(socar02)

In [34]:
# 이미지를 16x16 크기의 평균 해쉬로 변환
def img2hash(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#    gray = cv2.resize(gray, (600, 300)) # 이미지 croped & resize된 사진을 받음
    avg = gray.mean()
    bi = 1 * (gray > avg)
    return bi

# 해밍거리 측정 함수
def hamming_distance(a, b):
    a = a.reshape(1,-1)
    b = b.reshape(1,-1)
    # 같은 자리의 값이 서로 다른 것들의 합
    distance = (a !=b).sum()
    return distance

def image_match(image, stored_hash):
    # 들어온 차량의 이미지
    income_car_image = image    
    # 차 영상의 해쉬 구하기
    query_hash = img2hash(income_car_image)
    
    # 해당 차량의 저장된 해쉬 불러오기
    stored_hash = stored_hash
    
    # 해밍 거리 산출
    flag = 0
    hamming_dst = hamming_distance(query_hash, stored_hash)
    if hamming_dst/256 < 200: flag=1; print('이미지매칭: 동일 차량입니다.')
    else: print('동일하지 않은 차량입니다.')
        
    return income_car_image, flag

In [38]:
print(image_match(greencar01, greencar01_hash)[1])
print(image_match(greencar01, greencar02_hash)[1])
print(image_match(greencar01, socar01_hash)[1])
print(image_match(greencar01, socar02_hash)[1])
print(image_match(greencar02, socar01_hash)[1])
print(image_match(greencar02, socar02_hash)[1])

동일 차량입니다.
1
동일 차량입니다.
1
동일하지 않은 차량입니다.
0
동일하지 않은 차량입니다.
0
동일하지 않은 차량입니다.
0
동일하지 않은 차량입니다.
0


# 2. ORB(Oriented and Rotated BRIEF)와 knnMatch
 knnMatch() 함수는 디스크립터당 k개의 최근접 이웃 매칭점을 가까운 순서대로 반환합니다. k개의 최근접 이웃 중 거리가 가까운 것은 좋은 매칭점이고, 거리가 먼 것은 좋지 않은 매칭점일 가능성이 높습니다. 최근접 이웃 중 거리가 가까운 것 위주로 골라내면 좋은 매칭점을 찾아낼 수 있습니다.

In [42]:
# 서술자 추출 함수
def get_desc(img):
    img = img
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kp, desc = detector.detectAndCompute(gray, None)
    return kp, desc

In [40]:
# ORB로 서술자 추출 
detector = cv2.ORB_create()

# BF-Hamming 생성
matcher = cv2.BFMatcher(cv2.NORM_HAMMING2)

def ORB_knnMatch(income_car_image, flag, stored_kp, stored_desc):
    # 각 영상에 대해 키 포인트와 서술자 추출 
    compare_kp, compare_desc = get_desc(income_car_image)
    # knnMatch, k=2
    matches = matcher.knnMatch(compare_desc, stored_desc, 2)

    # 첫번재 이웃의 거리가 두 번째 이웃 거리의 75% 이내인 것만 추출---⑤
    ratio = 0.75
    good_matches = [first for first,second in matches \
                        if first.distance < second.distance * ratio]
    if len(good_matches) < 70: print('특징점 비교: 동일 차량입니다.')
    else: print('동일하지 않은 차량입니다')
    #print('matches:%d/%d' %(len(good_matches),len(matches)))

    # 좋은 매칭만 그리기
    #res = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, \
     #                   flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
    # 결과 출력
    #plt.figure(figsize=(15, 10))
    #plt.imshow(res)
    #plt.show()

In [52]:
greencar01_kp, greencar01_desc = get_desc(greencar01)
greencar02_kp, greencar02_desc = get_desc(greencar02)
socar01_kp, socar01_desc = get_desc(socar01)
socar02_kp, socar02_desc = get_desc(socar02)

In [53]:
# 이미지 매칭
income_car_image, flag = image_match(greencar01, greencar01_hash)

동일 차량입니다.


In [55]:
# 특징점 매칭
print(ORB_knnMatch(income_car_image, flag, greencar01_kp, greencar01_desc))
print(ORB_knnMatch(income_car_image, flag, greencar02_kp, greencar02_desc))
print(ORB_knnMatch(income_car_image, flag, socar01_kp, socar01_desc))
print(ORB_knnMatch(income_car_image, flag, socar02_kp, socar02_desc))


특징점 비교: 동일 차량입니다.
None
특징점 비교: 동일 차량입니다.
None
동일하지 않은 차량입니다
None
동일하지 않은 차량입니다
None
